<a href="https://colab.research.google.com/github/rehanbchinoy/Math-156-project/blob/main/ulysses_remake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

import base64
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#read the txt file to str
master = "https://raw.githubusercontent.com/rehanbchinoy/Math-156-project/main/ulysses_james_joyce.txt"
req = requests.get(master)
ulysses = req.text

In [ ]:
len(ulysses)

1552235

In [ ]:
#unique characters in the file
vocab = sorted(set(ulysses))
print(f'{len(vocab)} unique characters')

121 unique characters


In [ ]:
#all the characters appeared in Ulysses
#vocab

## Process the Text

### Vectorize

In [ ]:
#function that converts character tokens to numeric ids
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [ ]:
#function that converts numeric ids to character tokens
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
#function that combines character tokens back to strings
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Creating training examples and targets

In [ ]:
#convert ulysses from strings to numeric vector
all_ids = ids_from_chars(tf.strings.unicode_split(ulysses, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1552235,), dtype=int64, numpy=array([114,   3,  36, ...,  13,   2,   1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
#verify the ids dataset is correct
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

—
 
I
 
—









[
 
1
 
]






S
t
a
t
e
l
y
,
 
p
l
u
m
p
 
B
u
c
k
 
M
u
l
l
i
g
a
n
 
c
a
m
e
 
f
r
o
m
 
t
h
e
 
s
t
a
i
r
h
e
a
d
,
 
b
e
a
r
i
n
g
 
a
 
b
o
w
l
 
o
f



l
a
t
h
e
r
 


In [ ]:
#split the text into sequences with length 100
seq_length = 100
examples_per_epoch = len(ulysses)//(seq_length+1)

In [ ]:
#create the sequences based on sequences with length 101
#(it's 101 bc we need to split it into training and target sequences)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
#verify the sequences are created correctly
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xe2\x80\x94' b' ' b'I' b' ' b'\xe2\x80\x94' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'[' b' ' b'1' b' ' b']' b'\r' b'\n' b'\r' b'\n' b'S' b't'
 b'a' b't' b'e' b'l' b'y' b',' b' ' b'p' b'l' b'u' b'm' b'p' b' ' b'B'
 b'u' b'c' b'k' b' ' b'M' b'u' b'l' b'l' b'i' b'g' b'a' b'n' b' ' b'c'
 b'a' b'm' b'e' b' ' b'f' b'r' b'o' b'm' b' ' b't' b'h' b'e' b' ' b's'
 b't' b'a' b'i' b'r' b'h' b'e' b'a' b'd' b',' b' ' b'b' b'e' b'a' b'r'
 b'i' b'n' b'g' b' ' b'a' b' ' b'b' b'o' b'w' b'l' b' ' b'o' b'f' b'\r'
 b'\n' b'l' b'a' b't' b'h' b'e' b'r' b' ' b'o'], shape=(101,), dtype=string)


In [ ]:
#generate the target sequence for each sequence
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

### Create Training Batches

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
 
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try The Model

Run the model to see if the output is as expected

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 122) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  31232     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  125050    
                                                                 
Total params: 4,094,586
Trainable params: 4,094,586
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'larged glands, mumps,\r\nquinsy, bunions, hayfever, bedsores, ringworm, floating kidney,\r\nDerbyshire n'

Next Char Predictions:
 b'k&)9\xc3\xaerLD&,q\xc3\xbcEm\xc3\xb3\xc4\x81\xc3\xa6TZ\xc3\xbapP1FO\xc3\xb2Jr\xe2\x80\xa0\xc3\xae9Ut&sq\xe2\x80\xa2(jC\xc3\x80\xc3\xa4\xe2\x80\xa2\xe2\x80\xa0Q\xe2\x9c\xa0fik-\xc3\xb42(\xc2\xb0i &uB2\xc5\xbf\xc3\x86\xc4\x81\xc3\x9c0\xe2\x80\xa0\xc3\xbc\xc3\x865\xc3\x89.5\xc3\xbcR\xc3\x893fY\xc3\xb3\xc3\x9c\r7\xc3\xb3SqT\xe2\x80\x9d\xc5\x93lnQT?g\xe2\x80\xa61\xc2\xb0\xc4\x81Ya'


## Train The Model

### Add optimizer and loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 122)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.8036766


Check to make sure the loss is roughly the same as the vocabulary size

In [ ]:
tf.exp(mean_loss).numpy()

121.957985

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks'
# Name of the checkpoint files
#checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
240/240 [==============================] - 36s 136ms/step - loss: 2.6122
Epoch 2/20
240/240 [==============================] - 34s 136ms/step - loss: 2.0159
Epoch 3/20
240/240 [==============================] - 34s 136ms/step - loss: 1.7965
Epoch 4/20
240/240 [==============================] - 34s 135ms/step - loss: 1.6618
Epoch 5/20
240/240 [==============================] - 34s 135ms/step - loss: 1.5712
Epoch 6/20
240/240 [==============================] - 34s 135ms/step - loss: 1.5024
Epoch 7/20
240/240 [==============================] - 34s 135ms/step - loss: 1.4451
Epoch 8/20
240/240 [==============================] - 34s 135ms/step - loss: 1.3930
Epoch 9/20
240/240 [==============================] - 34s 135ms/step - loss: 1.3435
Epoch 10/20
240/240 [==============================] - 34s 135ms/step - loss: 1.2947
Epoch 11/20
240/240 [==============================] - 34s 134ms/step - loss: 1.2438
Epoch 12/20
240/240 [==============================] - 34s 134ms/step - lo

In [ ]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/checkpoint_epoch20')

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/ulysses_saved_model_epoch20')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/ulysses_saved_model_epoch20/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/ulysses_saved_model_epoch20/assets


## Generate Text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Duke'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Duke mammad
no hurry to throw for, throb in his mina border. What do they invent out
too downs what means to keep them to defull to think of home shewness.

—Who is he knew, Stephen interfected his armpit, Greem house. Two and
pinstrite of it. It was dear gord.

—He knows the speech, Mr O’Madden Burke said.

—You’re not serent as hurried simply. That was her coying born for their teeth
cold not her nostrils.

—Both of his feet.

Hoopewryt. Tommy was watching mustaritice. The Royal Davath Eggs. Waltz.
Acil’s words are girls. One please. Time that could he say. She got
a hackney came to Dublin. With a cough knocking over and he say for you. Call it Alexan
at their eyes would tempting manking armour? Are yested on afraid of
blooded mass. Tranquility beside the accumulous. Come in to them, he had
outlioner you: mind behind a kind of a moment. Bowing themselves in
that her church dissolse in the laugh but behind the dog after them, to set us
     God’s cifference withy Henr 

______________